In [1]:
import networkx as nx
import pickle
import glob
import copy
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from scipy.spatial.distance import pdist, squareform
from scipy.spatial import KDTree

import sys
sys.path.append("..")

In [2]:
camera_matrix = pickle.load(open("../camera_calibration/parameters/ir/camera_matrix.pkl", "rb"))
pose_graph = pickle.load(open("pose_graph.pkl", "rb"))
map_points = pickle.load(open("map_points.pkl", "rb"))

In [3]:
pose_graph

In [4]:
def get_neighbors(pose_graph, node_id):
    """Returns the node_ids of direct neighbors of the node specified by node_id."""
    neighbors_keyframes = [node_id for _, node_id in sorted(
        pose_graph.edges(newest_node_id))]
    return neighbors_keyframes

In [5]:
newest_node_id = list(sorted(pose_graph.nodes))[-1]
print("Bundle adjustment for keyframe {}".format(newest_node_id))

# get node_ids of neighboring keyframes
neighbors_keyframes = get_neighbors(pose_graph, newest_node_id)
print("Neighboring keyframes: {}".format(neighbors_keyframes))

nodes = [*neighbors_keyframes, newest_node_id]

Bundle adjustment for keyframe 8
Neighboring keyframes: [3, 4, 5, 6, 7]


In [12]:
sorted(list(pose_graph.neighbors(newest_node_id)))

[3, 4, 5, 6, 7]

In [6]:
[list(pose_graph.neighbors(n)) for n in pose_graph.neighbors(newest_node_id)]

[[6, 2, 3, 4, 5, 8],
 [2, 0, 1, 4, 5, 6, 7, 8],
 [3, 0, 1, 2, 5, 6, 7, 8],
 [4, 0, 1, 2, 3, 6, 7, 8],
 [5, 0, 1, 2, 3, 4, 7, 8]]

In [33]:
# We want to perform local BA with [3, 4, 5, 6, 7] + [8]
# We also want to include direct neighbors of all those neighbors as fixed vertices

def second_neighbors(graph, node):
    """Yield second neighbors of node in graph.
    Note: Neighbors are not not unique.
    """
    for neighbor_list in [graph.neighbors(n) for n in graph.neighbors(node)]:
        for n in neighbor_list:
            if (n != node) and (n not in graph.neighbors(node)):
                yield n
            
set(second_neighbors(pose_graph, newest_node_id))

{0, 1, 2}

In [9]:

def get_second_neighbors(pose_graph, node_id):
    """Returns second neighbors of node with node_id in pose graph."""
    second_neighbors = []
    for neighbor_list in [pose_graph.neighbors(n) for n in pose_graph.neighbors(node_id)]:
        for n in neighbor_list:
            if (n != node_id) and (n not in pose_graph.neighbors(node_id)):
                second_neighbors.append(n)
    return list(set(second_neighbors))
            
get_second_neighbors(pose_graph, newest_node_id)

[0, 1, 2]